In [10]:
# YOU HAVE TO SOURCE THE lattice/lattice/bin/activate virtual env for this, not conda!

In [11]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import tempfile

import sys
!{sys.executable} -m pip install pyarrow

import pandas as pd
import six
import tensorflow as tf
import tensorflow_lattice as tfl
import timeit
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import sys
sys.path.append("./lattice/lib/python3.6/site-packages")
import pyarrow
print(pyarrow.__path__)

['/local/ssd/home/mattfel/slac/timetool-ML-models/lattice/lattice/lib/python3.6/site-packages/pyarrow']


In [12]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 70m; }</style>"))

In [13]:

flags = tf.flags
FLAGS = flags.FLAGS

# Train example: python train.py --create_quantiles --output_dir=evaluations/output --quantiles_dir=evaluations/ --target=loadCycs --learning_rate=1 --train_epochs=100
# Test example: python train.py --output_dir=evaluations/output --quantiles_dir=evaluations/ --target=loadCycs

# test and train data set paths
project_dir = "./data/"
flags.DEFINE_string("train", project_dir + "train", "Path to test file.")
flags.DEFINE_string("test", project_dir + "test", "Path to train file.")

# Run mode of the program.
flags.DEFINE_string(
    "run", "train", "One of 'train', 'evaluate', 'time' or 'save', train will "
    "train on training data and also optionally evaluate; evaluate will "
    "evaluate train and test data; save saves the trained model so far "
    "so it can be used by TensorFlow Serving.")

# Model flags.

flags.DEFINE_string(
    "model_type", "calibrated_lattice",
    "Types defined in this example: calibrated_linear, calibrated_lattice, "
    " calibrated_lattice, calibrated_etl, calibrated_dnn")
flags.DEFINE_string("hparams", None,
                    "Model hyperparameters, see hyper-parameters in Tensorflow "
                    "Lattice documentation. Example: --hparams=learning_rate="
                    "0.1,lattice_size=2,num_keypoints=100")


# Calibration quantiles flags.
QUANTILES_DIR='./quantiles'
OUTPUT_DIR='./output'
EPOCHS=50
BATCH_SIZE=100
L2_LAPLACIAN = 5.0e-4
L2_TORSION = 1.0e-4
OPTIMIZER = tf.train.AdamOptimizer
NUM_LATTICES = 1
NUM_KEYPOINTS = 8
LATTICE_SIZE = 4
LEARNING_RATE = 20

flags.DEFINE_bool(
    "train_evaluate_on_train", True,
    "If set, every 1/10th of the train_epochs runs an evaluation on the "
    "full train data.")
flags.DEFINE_bool(
    "train_evaluate_on_test", False,
    "If set, every 1/10th of the train_epochs runs an evaluation on the "
    "full test data.")


# Extra flags: 
#   label defaults to normal (i.e. a non-malicious packet)
#   rtl_seed is a random seed to intialize lattices   
flags.DEFINE_string("target", "delay", "")
flags.DEFINE_integer("rtl_seed", 337893, "")



DuplicateFlagError: The flag 'train' is defined twice. First from /local/ssd/home/mattfel/slac/timetool-ML-models/lattice/lattice/lib/python3.6/site-packages/ipykernel_launcher.py, Second from /local/ssd/home/mattfel/slac/timetool-ML-models/lattice/lattice/lib/python3.6/site-packages/ipykernel_launcher.py.  Description from first occurrence: Path to test file.

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
column_names = ['fileId', 'row','rising_idx','falling_idx','volume','rising_weight',
                'falling_weight', 'first_val', 'last_val', 'delay']
def filterBad(dataset):
    initial_len = len(dataset)
    dataset = dataset[(dataset['rising_idx'] != 0)]
    dataset = dataset[(dataset['falling_idx'] != 0)]
    dataset = dataset[(dataset['volume'] > 500)]
#     dataset = dataset[(dataset['first_val'] < 30)]
#     dataset = dataset[(dataset['last_val'] < 30)]
    final_len = len(dataset)
    print('Rejected %d points (%f%%)' % ((initial_len-final_len), (initial_len-final_len)/initial_len))
    return dataset

raw_dataset = pd.read_csv('../preprocessing/processed.csv', names=column_names,
                      na_values = "?", comment='\t',
                      sep=",", skipinitialspace=True)
raw_dataset = filterBad(raw_dataset)
# visualize_dataset = raw_dataset.sample(frac=0.01)
# sns.pairplot(visualize_dataset[column_names], diag_kind="kde")


dataset = raw_dataset.copy()
dataset.tail()

dataset = dataset.dropna()
dataset = dataset.drop(columns=['fileId', 'row'])
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')
# dataset.tail()
# train_dataset = dataset.sample(frac=0.8,random_state=0)
# test_dataset = dataset.drop(train_dataset.index)

# train_stats = train_dataset.describe()
# train_stats.pop("delay")
# train_stats = train_stats.transpose()
# train_stats
# train_labels = train_dataset.pop('delay')
# test_labels = test_dataset.pop('delay')

from sklearn.model_selection import train_test_split

X, y = dataset.iloc[:,:-1],dataset.iloc[:,-1]

train_dataset, test_dataset, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=123)





In [14]:
train_input_fn = tf.estimator.inputs.pandas_input_fn(
  x=train_dataset,
  y=train_labels,
  batch_size=BATCH_SIZE,
  shuffle=False,
  num_epochs=EPOCHS,
  num_threads=1)

test_input_fn = tf.estimator.inputs.pandas_input_fn(
  x=test_dataset,
  y=test_labels,
  batch_size=BATCH_SIZE,
  shuffle=False,
  num_epochs=1,
  num_threads=1)

In [16]:

trainable_features = ['rising_idx','falling_idx','volume','rising_weight',
                'falling_weight', 'first_val', 'last_val']

target = 'delay'

# Set up test and train functions with relevant parameters
def get_test_input_fn(batch_size, num_epochs, shuffle):
#   return get_input_fn(FLAGS.test, batch_size, num_epochs, shuffle)
    return test_input_fn

def get_train_input_fn(batch_size, num_epochs, shuffle):
#   return get_input_fn(FLAGS.train, batch_size, num_epochs, shuffle)
    return train_input_fn



# # Copy of data read from train/test files: keep copy to avoid re-reading
# # it at every training/evaluation loop.
# _df_data = {}
# _df_data_labels = {}



# # Load data from files
# def get_input_fn(file_path, batch_size, num_epochs, shuffle):
#   """Returns an input_fn closure for given parameters."""
#   if file_path not in _df_data:

#     # Load data in column_names format
#     print("Loading data from ", file_path)
#     _df_data[file_path] = pd.read_csv(
# #         tf.gfile.Open(file_path),
#         tf.gfile.Open('../preprocessing/processed_baby.csv'),
#         sep=',',
#         names=column_names,
#         skipinitialspace=True,
#         engine="python",
#         dtype=np.float32)
# #     _df_data[file_path].drop(columns=list(targets - {FLAGS.target}), errors="ignore", inplace=True)

#     # Mark labels
#     _df_data_labels[file_path] = (_df_data[file_path][FLAGS.target])

#   # set up data with labels
#   return tf.estimator.inputs.pandas_input_fn(
#       x=_df_data[file_path],
#       y=_df_data_labels[file_path],
#       batch_size=batch_size,
#       shuffle=shuffle,
#       num_epochs=num_epochs,
#       num_threads=1)


# Create feature columns with correct categorical vocabularies
def create_feature_columns():
  return [tf.feature_column.numeric_column(feat) for feat in trainable_features if feat != target]

# Create quantiles based on batch size
def create_quantiles(quantiles_dir):
  """Creates quantiles directory if it doesn't yet exist."""
  batch_size = 100
  print("creating quantiles...")
  input_fn = get_train_input_fn(
      batch_size=batch_size, num_epochs=1, shuffle=False)
  # Reads until input is exhausted, 50 at a time.
  tfl.save_quantiles_for_keypoints(
      input_fn=input_fn,
      save_dir=quantiles_dir,
      feature_columns=create_feature_columns(),
      num_steps=None)


# Print hyper parameters
def _pprint_hparams(hparams):
  """Pretty-print hparams."""
  print("* hparams=[")
  for (key, value) in sorted(six.iteritems(hparams.values())):
    print("\t{}={}".format(key, value))
  print("]")


# # Create network
# def create_network(model_fn1, model_fn2, feature_columns, config, quantiles_dir):
#   """Creates a calibrated Lattice estimator."""
#   """ Lattice 0 (Competitors) """
#   feature_names = [fc.name for fc in feature_columns if fc not in payloads]
#   hparams = tfl.CalibratedLatticeHParams(
#       feature_names=feature_names,
#       learning_rate=FLAGS.learning_rate,
#       lattice_l2_laplacian_reg=5.0e-4,
#       # lattice_l2_torsion_reg=1.0e-4,
#       interpolation_type='hypercube',
#       num_keypoints=8,
#       lattice_size=5,
#       lattice_rank=len(feature_names),
#       num_lattices=1,
#       optimizer=tf.train.AdamOptimizer)
#   hparams.parse(FLAGS.hparams)
#   _pprint_hparams(hparams)
#   (layer00, _, _, _) = tfl.calibrated_lattice_regressor(
#     model_fn1,
#     feature_columns=feature_columns,
#     model_dir=config.model_dir,
#     config=config,
#     quantiles_dir=quantiles_dir,
#     keypoints_initializers_fn=None,
#     optimizer=tf.train.AdamOptimizer,
#     hparams=hparams
#   )

#   """ Lattice 1 (Transfer info) """
#   feature_names = [fc.name for fc in feature_columns if fc not in competitors]
#   hparams = tfl.CalibratedLatticeHParams(
#       feature_names=feature_names,
#       learning_rate=FLAGS.learning_rate,
#       lattice_l2_laplacian_reg=5.0e-4,
#       # lattice_l2_torsion_reg=1.0e-4,
#       interpolation_type='hypercube',
#       num_keypoints=8,
#       lattice_size=16,
#       lattice_rank=len(feature_names),
#       num_lattices=1,
#       optimizer=tf.train.AdamOptimizer)
#   hparams.parse(FLAGS.hparams)
#   _pprint_hparams(hparams)
#   (layer01, _, _, _) = tfl.calibrated_lattice_regressor(
#     model_fn2,
#     feature_columns=feature_columns,
#     model_dir=config.model_dir,
#     config=config,
#     quantiles_dir=quantiles_dir,
#     keypoints_initializers_fn=None,
#     optimizer=tf.train.AdamOptimizer,
#     hparams=hparams
#   )

#   """ Lattice 2 (combine lattices 0 and 1) """
#   feature_names = ['lattice0','lattice1']
#   hparams = tfl.CalibratedLatticeHParams(
#       feature_names=feature_names,
#       learning_rate=FLAGS.learning_rate,
#       lattice_l2_laplacian_reg=5.0e-4,
#       # lattice_l2_torsion_reg=1.0e-4,
#       interpolation_type='hypercube',
#       num_keypoints=8,
#       lattice_size=16,
#       lattice_rank=len(feature_names),
#       num_lattices=1,
#       optimizer=tf.train.AdamOptimizer)
#   hparams.parse(FLAGS.hparams)
#   _pprint_hparams(hparams)
#   layer10 = tfl.calibrated_lattice_regressor(
#     feature_columns=feature_columns,
#     model_dir=config.model_dir,
#     config=config,
#     quantiles_dir=quantiles_dir,
#     keypoints_initializers_fn=None,
#     optimizer=tf.train.AdamOptimizer,
#     hparams=hparams
#   )

#   layer10


# Create a set of randomly initialized lattices with calibrator inputs
def create_calibrated_lattice(feature_columns, config, quantiles_dir):
  """Creates a calibrated Lattice estimator."""
  feature_names = [fc.name for fc in feature_columns]
  hparams = tfl.CalibratedLatticeHParams(
      feature_names=feature_names,
      learning_rate=LEARNING_RATE,
      lattice_l2_laplacian_reg=L2_LAPLACIAN, 
      lattice_l2_torsion_reg=L2_TORSION,
      interpolation_type='simplex',
      num_keypoints=NUM_KEYPOINTS, 
#       feature__outerIters__monotonicity=1,
#       feature__innerIters__monotonicity=1,
      # feature__bitsPerCycle__monotonicity=-1,
      # feature__innerIters__lattice_size=3,
      # feature__outerIters__lattice_size=3,
      # monotonicity=frozenset({'outerIters': 1}.items),
      lattice_size=LATTICE_SIZE, 
      lattice_rank=len(feature_columns),
      num_lattices=NUM_LATTICES, 
      optimizer=OPTIMIZER) 

  # Specific feature parameters.
  hparams.parse(FLAGS.hparams)
  _pprint_hparams(hparams)

  return tfl.calibrated_lattice_regressor(
    feature_columns=feature_columns,
    model_dir=config.model_dir,
    config=config,
    quantiles_dir=quantiles_dir,
    keypoints_initializers_fn=None,
    optimizer=tf.train.AdamOptimizer,
    hparams=hparams
  )





# Create an estimator
# TODO - Add other lattice models here
def create_estimator(config, quantiles_dir):
  """Creates estimator for given configuration based on --model_type."""
  feature_columns = create_feature_columns()
  #FLAGS.model_type == "calibrated_rtl":
  return create_calibrated_lattice(feature_columns, config, quantiles_dir)

  #raise ValueError("Unknown model_type={}".format(FLAGS.model_type))



# Evaluator that keeps track accuracy and loss
def evaluate_on_data(estimator, data):
  """Evaluates and prints results, set data to FLAGS.test or FLAGS.train."""
  name = os.path.basename(data)
  evaluation = estimator.evaluate(
      input_fn=get_input_fn(
          file_path=data,
          batch_size=BATCH_SIZE,
          num_epochs=1,
          shuffle=False),
      name=name)


  metrics = [
          "average_loss"
          ]
  print(estimator)
  print(evaluation)
  metric_string = "\t".join("{}={:.8f}".format(metric, evaluation[metric]) for metric in metrics)
  print(metric_string)

  return evaluation
  #print("  Evaluation on '{}':\taccuracy={:.4f}\taverage_loss={:.4f}".format(
      #name, evaluation["accuracy"], evaluation["average_loss"]))

# Training function
def train(estimator):
  """Trains estimator and optionally intermediary evaluations."""
  if not FLAGS.train_evaluate_on_train and not FLAGS.train_evaluate_on_test:
    estimator.train(input_fn=get_train_input_fn(
        batch_size=BATCH_SIZE,
        num_epochs=EPOCHS,
        shuffle=True))
  else:
    # Train 1/10th of the epochs requested per loop, but at least 1 per loop.
    epochs_trained = 0
    loops = 0
    while epochs_trained < EPOCHS:
      loops += 1
      next_epochs_trained = int(loops * EPOCHS / 10.0)
      epochs = max(1, next_epochs_trained - epochs_trained)
      epochs_trained += epochs
      inp = get_train_input_fn(batch_size=BATCH_SIZE, num_epochs=epochs, shuffle=True)
      estimator.train(input_fn=inp)
      print("Trained for {} epochs, total so far {}:".format(
          epochs, epochs_trained))
      if FLAGS.train_evaluate_on_train:
          evaluate_on_data(estimator, FLAGS.train)
      if FLAGS.train_evaluate_on_test:
          evaluate_on_data(estimator, FLAGS.test)


# Train before testing
def evaluate(estimator):
  """Runs straight evaluation on a currently trained model."""
  evaluate_on_data(estimator, FLAGS.train)
  evaluate_on_data(estimator, FLAGS.test)

def timed_evaluate(estimator):

  def eval_func():
    evaluate_on_data(estimator, FLAGS.test)


  """ Timed evaluation over 50 trials"""
  runtime = timeit.timeit(eval_func, number = 3)*1000000
  sperinf = (runtime / float(10000))
  print("%f us/inference (%fus / %d)" % (sperinf, runtime, 10000))

# Main function that sets up and runs program
def main(args):
  del args  # Not used.

  # Prepare directories.
  output_dir = OUTPUT_DIR
  if output_dir is None:
    output_dir = tempfile.mkdtemp()
    tf.logging.warning("Using temporary folder as model directory: %s",
                       output_dir)
  quantiles_dir = QUANTILES_DIR or output_dir

  # Create quantiles if required.
  if CREATE_QUANTILES:
    if FLAGS.run != "train":
      raise ValueError(
          "Can not create_quantiles for mode --run='{}'".format(FLAGS.run))
    create_quantiles(quantiles_dir)

  # Create config and then model.
  config = tf.estimator.RunConfig().replace(model_dir=output_dir)
  estimator = create_estimator(config, quantiles_dir)

  if FLAGS.run == "train":
    train(estimator)

  elif FLAGS.run == "evaluate":
    evaluate(estimator)
    
  elif FLAGS.run == "time":
    timed_evaluate(estimator)

  else:
    raise ValueError("Unknown --run={}".format(FLAGS.run))





In [17]:
if __name__ == "__main__":
  FLAGS.run = "train"
  CREATE_QUANTILES=True
  tf.app.run()
  print("quantiles created!")
    
  CREATE_QUANTILES=False
  tf.app.run()
    
    


creating quantiles...


ValueError: Unbound (num_steps=None) materialization of input reached safety size of 1000000000.0

In [ ]:
rmse = mean_squared_error(y_test, preds)
rmae = mean_absolute_error(y_test, preds)
r2 = r2_score(y_test, preds)
print("MSE, MAE, r2: %f,%f,%f" % (rmse, rmae, r2))